In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import json
import os
import time
import string
import random
import os.path
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [3]:
# set path WTO_data_article
path = os.path.abspath('../WTO_data_article')
print(path)

/Users/ycchen/voxgov/WTO_crawler/WTO_data_article


In [3]:
all_urls = set()
for folder in os.listdir(path):
    print(folder)
    sub_urls = set()

    for file in os.listdir(path + '/' + folder):
        # print(file)
        data = pd.read_csv(path + '/' + folder + '/' + file)
        sub_urls = sub_urls.union(set(data['url']))
        all_urls = all_urls.union(set(data['url']))
    print('category unique:', len(sub_urls))
print('total unique:', len(all_urls))

all_article_content
category unique: 0
News_archives
category unique: 9842
.DS_Store


NotADirectoryError: [Errno 20] Not a directory: '/Users/ycchen/voxgov/WTO_crawler/WTO_data_article/.DS_Store'

In [118]:
all_urls = set()
for folder in os.listdir(path):
    print(folder)
    # sub_urls = set()

    # for file in os.listdir(path + '/' + folder):
    #     if '.DS_Store' not in file:
    #         print(file)

all_article_content
News_archives
.DS_Store
Subject_archives
Press_releases
DG_speeches


In [8]:
url = 'https://www.wto.org/english/blogs_e/ddg_anabel_gonzalez_e/blog_ag_06dec21_e.htm'
options = Options()
options.headless = True
driver = webdriver.Firefox(executable_path=r'../geckodriver', options=options)
driver.get(url)

In [22]:
title = driver.find_element(By.ID, 'blogtitle').text
abstract = None
label = None
date = driver.find_element(By.ID, 'blogpostdate').text
content = driver.find_element(By.ID, 'blogtext').text

In [25]:
[a.get_attribute('href') for a in driver.find_element(By.ID, 'blogtext').find_elements(By.TAG_NAME, 'a')]

['https://www.reuters.com/business/wto-seeks-deals-early-2022-despite-postponing-major-meeting-2021-11-30/',
 'https://docs.wto.org/dol2fe/Pages/SS/directdoc.aspx?filename=q:/INF/SDR/1.pdf&Open=True',
 'https://www.wto.org/english/news_e/news21_e/jssdr_26nov21_e.htm',
 'https://www.wto.org/english/news_e/news21_e/ita_02dec21_e.htm',
 'https://www.wto.org/english/tratop_e/gproc_e/gp_gpa_e.htm',
 'https://docs.wto.org/dol2fe/Pages/SS/directdoc.aspx?filename=q:/WT/MIN21/W5.pdf&Open=True',
 'https://docs.wto.org/dol2fe/Pages/SS/directdoc.aspx?filename=q:/WT/MIN21/W5A1.pdf&Open=True',
 'https://www.wto.org/english/news_e/news21_e/hod_02dec21_e.htm']

In [8]:
label

'WTO PUBLIC FORUM'

In [132]:
outboundLinks = [a.get_attribute('href') for a in driver.find_element(By.CLASS_NAME, 'centerCol').find_elements(By.TAG_NAME, 'a')]
outboundLinks

['https://www.wto.org/english/forums_e/public_forum16_e/public_forum16_e.htm',
 'mailto:pf16@wto.org']

In [130]:
for outboundLink in outboundLinks[:-5]:
    if 'javascript' in outboundLink:
        linkdoldoc = re.search(r"'(.*?)'", outboundLink)[1]
        outboundLink = f'https://docs.wto.org/dol2fe/Pages/SS/directdoc.aspx?filename=q:{linkdoldoc}&Open=True'
    print(outboundLink)

In [111]:
def extract_linkdoldoc(outboundLink):
    if 'javascript' in outboundLink:
        linkdoldoc = re.search(r"'(.*?)'", outboundLink)[1]
        outboundLink = f'https://docs.wto.org/dol2fe/Pages/SS/directdoc.aspx?filename=q:{linkdoldoc}&Open=True'
    return outboundLink


outboundLinks = [extract_linkdoldoc(outboundLink) for outboundLink in outboundLinks[:-5]]

In [131]:
outboundLinks

['https://www.wto.org/english/forums_e/public_forum16_e/public_forum16_e.htm',
 'mailto:pf16@wto.org']

In [135]:
outboundLinksText = [a.text for a in driver.find_element(By.CLASS_NAME, 'centerCol').find_elements(By.TAG_NAME, 'a')]

In [136]:
outboundLinksText

['www.wto.org/pf16', 'pf16@wto.org']

In [126]:
articleDict = {url}
articleDict[url].update({'date': date, 
                    'label': label, 
                    'abstract': abstract, 
                    'title': title, 
                    'content': content, 
                    'outboundLinks': outboundLinks, 
                    'outboundLinksText': outboundLinksText})
articleDict

TypeError: 'set' object is not subscriptable

In [5]:

centerCol = driver.find_element(By.CLASS_NAME, 'centerCol')
trs = centerCol.find_elements(By.TAG_NAME, 'tr')

In [6]:
df = []

for tr in trs:
    for td in tr.find_elements(By.TAG_NAME, 'td'):
        for x in td.find_elements(By.TAG_NAME, 'h3'):
            # if len(x.text) > 8:
            title = x.text
            print(title)
        for link in td.find_elements(By.TAG_NAME, 'a'):
            type = link.text
            url = link.get_attribute('href')
            df.append([title, type, url])
            
df = pd.DataFrame(df, columns=['title', 'type', 'url'])

Sweden donates SEK 19.2 million to help developing countries participate in global trade
EU gives EUR 1 million to enhance trading skills of developing countries
Lithuania donates EUR 26,000 to enhance trading capacity of developing countries
Finland donates EUR 1.7 million to help developing countries participate in global trade
France donates EUR 50,000 to support food safety, animal/plant health standards and trade
United Kingdom donates GBP 100,000 to support LDCs’ participation in Ministerial Conference
Norway donates NOK 10 million to help developing countries participate in global trade
Canada donates CAD 100,000 to support LDCs’ participation in Ministerial Conference
United States donates USD 600,000 to enhance negotiating skills of developing countries
Liechtenstein donates CHF 40,000 to enhance negotiating skill-set of developing countries
Sweden donates SEK 500,000 to support LDCs’ participation in Ministerial Conference
Republic of Korea gives USD 350,000 to help developin

In [9]:
title

'Troika recommends Carvalho de Azevêdo to be the next WTO Director-General'

In [119]:
if not os.path.exists(f'../WTO_data_article/all_article_content.json'):
    with open(path + '/all_article_content.json', 'w') as fp:
        json.dump({}, fp)
    fp.close()
else:
    with open(path + '/all_article_content.json', 'r') as fp:
        all_article_content = json.load(fp)
        print(f'there are already {len(all_article_content)} articles in the json file')
        print(f'{len(all_urls) - len(all_article_content)} articles to go')
    fp.close()

there are already 0 articles in the json file
0 articles to go


In [121]:
all_article_content.keys()

dict_keys([])

In [122]:
for url in all_urls:
    if url in all_article_content.keys() or url == None:
        continue
    else:
        print(url)

In [123]:
all_urls

set()

In [137]:
# print current time
print(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()))

2023-12-20 20:59:15


In [24]:
# print a random number from 1 to 5
print(random.randint(1, 5))

1


In [46]:
# open the browser and get the page
options = Options()
options.headless = True
driver = webdriver.Firefox(executable_path=r'../geckodriver', options=options)

main_url = 'https://www.wto.org/english/news_e/news_e.htm#'
driver.get(main_url)

In [50]:
TreeviewSpanArea = driver.find_elements(By.CLASS_NAME, 'TreeviewSpanArea')[1]

In [70]:
targetMenuLst = ['News archives', 'Press releases', 'DG speeches', 'Subject archives']

<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="8911b63f-1a69-4445-a81f-85497d2de597", element="d3503f5e-f35a-401c-8f37-8db4a65bface")>

In [63]:
subMenu = TreeviewSpanArea.find_elements(By.TAG_NAME, 'a')

In [72]:
subMenuText = [subMenuText.get_attribute('text') for subMenuText in subMenu]
subMenuUrl = [subMenuUrl.get_attribute('href') for subMenuUrl in subMenu]

In [80]:
# targetMenuUrlDict = dict(zip(subMenuText, subMenuUrl))
targetMenuUrlDict = {}
targetMenu_idx = 0
idx = 0

for url in subMenuUrl:
    if url == main_url:
        targetMenu = targetMenuLst[targetMenu_idx]
        targetMenuUrlDict.update({targetMenu:{}})
        targetMenu_idx += 1
        idx += 1
        continue
    
    targetMenuUrlDict[targetMenu].update({subMenuText[idx]: subMenuUrl[idx]})
    idx += 1

In [77]:
targetMenuUrlDict = {}
targetMenuUrlDict.update({targetMenuLst[0]:{}})

In [83]:
targetMenuUrlDict['Subject archives'].keys()

dict_keys(['25th anniversary of the WTO', 'Accessions', 'Agriculture', 'Agriculture negotiations', 'Aid for Trade', 'Aid for Trade Global Review 2019', 'Aid for Trade Global Review 2022', 'Anti-dumping', 'Appellate Body', 'Balance of payments', 'Business, trade and the WTO', 'Climate change', 'COP27', 'COP28', 'Cotton', 'Council for Trade in Goods', 'COVID-19 and world trade', 'Customs valuation', 'Deputy Director-General Angela Ellard', 'Deputy Director-General Jean-Marie Paugam', 'Deputy Director-General Johanna Hill', 'Deputy Director-General Xiangchen Zhang', 'Deputy Directors-General', 'Deputy Directors-General under Pascal Lamy', 'Deputy Directors-General under Roberto Azevêdo', 'Dialogue on plastics pollution and environmentally sustainable plastics trade', 'Digital technologies and trade', 'Director-General Ngozi Okonjo-Iweala', 'Director-General selection process', 'Dispute settlement', 'Dispute Settlement Body', 'Doha Development Agenda', 'Economic research', 'Electronic comm